### Scraping abstract information
March 4, 2018
This notebook scrapes abstract text from:
- Proceedings of the Annual Cognitive Science Society meeting archive (html)
- Proceedings of Cognitive Neuroscience Society annual meeting (text converted from pdf)

Abstracts are then stored in a spreadsheet, containing information such as year, authors, title, and abstract.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import csv
from bs4 import BeautifulSoup
import urllib
import re
import string
import pandas as pd
import os
import sys

In [2]:
def scrape_CS(home_url, data_file):
    #connect to home page url for that year
    CSurl = urllib.request.urlopen(home_url).read()
    soup = BeautifulSoup(CSurl, 'html.parser')
    all_links = soup.find_all('a', attrs={'href': re.compile("papers/*")})    
    year = home_url[-5:-1]    
    
    # enumerate through all paper links
    for link_idx, link in enumerate(all_links):
        # get soup from paper url
        if home_url not in str(link['href']):
            url_text = home_url + str(link['href'])
        else:
            url_text = str(link['href'])
    
        url = urllib.request.urlopen(url_text).read()
        soup = BeautifulSoup(url, 'html.parser')
    
        # scrape & parse
        authors = []
        affl = []
        title = ' '.join(soup.find_all('h1')[0].text.split())
        # exception rule for 2014 abstracts
        if '2014' in home_url:            
            abstr = ' '.join(soup.find_all('blockquote')[1].text.split())
        else:            
            abstr = ' '.join(soup.find_all('p', {"id": "abstract"})[0].text.split())            
        
        soup.find_all('ul')
        for ana in soup.find_all('em'):
            affl.append('>'+ana.text)
            if '2014' in home_url:
                # somebody fucked something up in 2014
                authors.append('>' + ana.previous_element.previous_element.split(',')[0])
            else:            
                authors.append('>' + ana.previous_element.split(',')[0])
        
        # do some gymnastics to get it into a pandas df and add as a row to CSV
        new_row = {'Year': str(year), 'Title': title,'Abstract': abstr,'Authors': ''.join(authors),'Affiliations': ''.join(affl), 'URL': url_text}
        df_cur = pd.Series(data=new_row).to_frame().T[['Year','Title','Abstract','Authors','Affiliations','URL']]
        df_cur.to_csv(data_file, mode='a', header=False, index=False)


In [ ]:
# get all paper links from cogsci conference
home_urls = ['https://mindmodeling.org/cogsci2017/',
             'https://mindmodeling.org/cogsci2016/',
             'https://mindmodeling.org/cogsci2015/',
             'https://mindmodeling.org/cogsci2014/',
             'https://mindmodeling.org/cogsci2013/',
             'https://mindmodeling.org/cogsci2012/',
             'https://mindmodeling.org/cogsci2011/',
             'https://mindmodeling.org/cogsci2010/']

for year in home_urls:
    # scrape all
    print(year)
    scrape_CS(home_url=year, data_file='../data/cogsci_abstracts.csv')

### gather CNS abstracts from text to csv

In [3]:
data_folder = '../data/CNS_programs/'
os.listdir(data_folder)
CNS_files = sorted([f for f in os.listdir(data_folder) if ('CNS' in f) and ('.txt' in f)])[:-1]
print(CNS_files)

['CNS_2007_Program.txt', 'CNS_2008_Program.txt', 'CNS_2009_Program.txt', 'CNS_2010_Program.txt', 'CNS_2011_Program.txt', 'CNS_2012_Program.txt', 'CNS_2013_Program.txt', 'CNS_2014_Program.txt', 'CNS_2015_Program.txt', 'CNS_2016_Program.txt']


In [4]:
#col_names = ['CNS_file','start_sect','beg_ind','end_ind']
CNS_file=[]
start_sect=[]
beg_ind=[]
end_ind=[]

for i in range(len(CNS_files)):
    
    CNS_file.append(CNS_files[i])
    
    #print(CNS_files[i])
    file = open(data_folder+CNS_files[i], 'r')
    data = file.read()
    data_list = data.split('\n')
    abs_start = [ind for ind, d in enumerate(data_list) if '\x0cPoster Session A' in d][0]#tells you the ind where the first abstract is?
    
    start_sect.append([ind for ind, d in enumerate(data_list) if '\x0cPoster Session A' in d])
    #print([ind for ind, d in enumerate(data_list) if '\x0cPoster Session A' in d])
    
    abs_list = data_list[abs_start:]
    poster_beg_ind = next((ind for ind,s in enumerate(abs_list) if '\x0cPoster Session A' == s), None)    
    poster_end_ind = next((ind for ind,s in enumerate(abs_list) if '\x0cAuthor Index' == s), None)
    
    
    beg_ind.append(poster_beg_ind)
    
    end_ind.append(poster_end_ind)
    
    
    #print(poster_beg_ind, poster_end_ind)

In [ ]:
cns_frame=pd.DataFrame({'CNS_file':CNS_file,'start_sect':start_sect,'beg_ind':beg_ind,'end_ind':end_ind})
cns_frame

In [8]:
sect_abs=[]
abst=[]
title=[]
auth=[]
i=1
letter_start_ind = data_list.index('\x0cPoster Session A')
letter_end_ind = data_list.index('\x0cPoster Session B')
a_b_range= range(letter_start_ind,letter_end_ind)
a_b_range=list(a_b_range)
file = open(data_folder+CNS_files[9], 'r')
data = file.read()
data_list = data.split('\n')
poster_beg_ind = data_list.index('\x0cPoster Session A')
#while cur_section = 'A':#dont know how this cur_section string being A will work. thinking 
    #better idea being from poster_beg_ind = data_list.index('\x0cPoster Session A') 
    #to poster_beg_ind = data_list.index('\x0cPoster Session B') make this a range
#['A' 'B']
#for cur_section in 
for i in range(len(a_b_range)):
    try:
        cur_section='A'
        cur_abs=i+1
        abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
        abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

        section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
    #     print(section_abst)
    #     print(cur_section+'%i'%(cur_abs+1))
    #     print(i, abs_beg_ind, abs_end_ind)
    except ValueError:
        print('number not found')
        try:
            start_abst=section_abst.index(' — ')#index with in the section where we first see this character - which denotes start of abs

            whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
            title_sect=section_abst[0:start_abst]#the title and author combined
            title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
            title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words
            length_title_lst=' '.join(title_lst)
            auth_sect=section_abst[len(length_title_lst):start_abst]


            sect_abs.append(cur_section+'%i'%cur_abs)
            abst.append(whole_abs)
            title.append(length_title_lst)
            #auth.append(title_sect) old wrong way of getting author
            auth.append(auth_sect)
        except :
            print(cur_section+'%i'%(cur_abs))
            
    

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

In [ ]:
a_b_range

In [ ]:
#this cell is the bare bones for how the function will run

#AKA test weird cases in here bc you can input the specific abstract you want

# d = {'sect_abs':[],'abs':[],'title':[],'auth':[]}
# Df_2016=pd.DataFrame(data=d)

# for i in range(len())
cur_section = 'A'
#cur_section = 'E '
cur_abs = 5
abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
start_abst=section_abst.index(" — ")#index with in the section where we first see this character - which denotes start of abs
whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
title_sect=section_abst[0:start_abst]#the title and author combined
title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words


length_title_lst=' '.join(title_lst)

auth_sect=section_abst[len(length_title_lst):start_abst]

# Df_2016.sect_abs.append(cur_section)
# Df_2016.abs.append(whole_abs)
# Df_2016.title.append(title_lst)
# Df_2016.auth.append(title_sect)
#WIP=[word for word in title_sect if not word.islower() and not word.isupper()]#this could possibly be a filter however I intended it to give me mixed case strings which would be the authors.
#WIP

In [ ]:
auth_sect

In [ ]:
length_title_lst

In [5]:
#making a datatable to see how the data fills the table which will help better understand the format i need to put the data into
try_table=pd.DataFrame({'sect_abs':sect_abs,'abst':abst,'title':title,'auth':auth})




NameError: name 'sect_abs' is not defined

Notes for table
-abst seems fine

-auth
    want to take out the A# and lowercase it and only include the actual author

-sect_abs
    want to add number to the A

-title
    want to join and then lower and drop the A#

In [ ]:
#for i in range(len(CNS_files)):
for i in range(1):
    file = open(data_folder+CNS_files[9], 'r')
    print(CNS_files[9])
    data = file.read()
    data_list = data.split('\n')
    poster_beg_ind = data_list.index('\x0cPoster Session A')
    if '\x0cPoster Topic Index' in data_list:
        poster_end_ind = data_list.index('\x0cPoster Topic Index')
    else:
        poster_end_ind = data_list.index('\x0cAuthor Index')

    #print(poster_beg_ind, poster_end_ind)
    ' '.join(abs_list[abs_beg_ind:abs_end_ind])
    #print(data_list[poster_beg_ind:poster_end_ind])
    #print(poster_end_ind)

In [ ]:
#print(CNS_files[1])
file = open(data_folder+CNS_files[9], 'r')
data = file.read()
data_list = data.split('\n')
print(data_list.index('\x0cPoster Session A'))
data_list[data_list.index('\x0cPoster Session H'):]
' '.join(data_list[data_list.index('\x0cPoster Session H'):])
#splitiing the data on the new line and then concatinating the list it makes with space and .join
#Good starting point
#try to follow format for the other csv that is sompleted this one is not completed
#do Not do 2017 because they do not have the abstracts, however if i can finish looping through all exacpt 2017
#then go back to 2017 and do the talks

#of the ones in CNS to do, they all seem to follow this format of having for example H1 and then the title of the 
#abstract and that is in all caps and then that is followed by the authors and they all have numbers 
#and then the abstract startsso there are some nice delimeters in there that i can work with

In [ ]:
def find_num_sess(abs_list, poster_beg_ind):
    # find the number of poster sessions from the schedule section
    sched = abs_list[:poster_beg_ind]
    for ind, sess in enumerate(string.ascii_uppercase):
        if sess not in sched:  
            return string.ascii_uppercase[:ind]

sess = find_num_sess(abs_list, poster_beg_ind)
sess
#'Poster Session ' + sess[0]



In [ ]:
whole_abs

In [ ]:
sect_abs=[]
abst=[]
title=[]
auth=[]
i=1
letter_start_ind = data_list.index('\x0cPoster Session A')
letter_end_ind = data_list.index('\x0cPoster Session B')
a_b_range= range(letter_start_ind,letter_end_ind)
a_b_range=list(a_b_range)
file = open(data_folder+CNS_files[9], 'r')
data = file.read()
data_list = data.split('\n')
poster_beg_ind = data_list.index('\x0cPoster Session A')
#while cur_section = 'A':#dont know how this cur_section string being A will work. thinking 
    #better idea being from poster_beg_ind = data_list.index('\x0cPoster Session A') 
    #to poster_beg_ind = data_list.index('\x0cPoster Session B') make this a range
#['A' 'B']
#for cur_section in 
for i in range(len(a_b_range)):
    cur_section='A'
    cur_abs=i+1
    abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
    abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

    section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
#     print(section_abst)
#     print(cur_section+'%i'%(cur_abs+1))
#     print(i, abs_beg_ind, abs_end_ind)
    try:
        start_abst=section_abst.index(' — ')#index with in the section where we first see this character - which denotes start of abs

        whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
        title_sect=section_abst[0:start_abst]#the title and author combined
        title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
        title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words
        length_title_lst=' '.join(title_lst)
        auth_sect=section_abst[len(length_title_lst):start_abst]
        
        
        sect_abs.append(cur_section+'%i'%cur_abs)
        abst.append(whole_abs)
        title.append(length_title_lst)
        #auth.append(title_sect) old wrong way of getting author
        auth.append(auth_sect)
    except :
        print(cur_section+'%i'%(cur_abs))

In [9]:
sect_abs=[]
abst=[]
title=[]
auth=[]
i=1
letter_start_ind = data_list.index('\x0cPoster Session A')
letter_end_ind = data_list.index('\x0cPoster Session F')
a_b_range= range(letter_start_ind,letter_end_ind)
#a_b_range=list(a_b_range)
alpha=['A','B','C','D','E','F']
file = open(data_folder+CNS_files[9], 'r')
data = file.read()
data_list = data.split('\n')
poster_beg_ind = data_list.index('\x0cPoster Session A')

#messing with letter iterations

big_range=letter_end_ind-letter_start_ind


#for cur_section in 
for letter in alpha:
    for i in range(big_range):
        try:
            #cur_section=alpha[letter]
            cur_abs=i+1
    #         abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
    #         abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))
            abs_beg_ind = abs_list.index(letter+'%i'%cur_abs)
            abs_end_ind = abs_list.index(letter+'%i'%(cur_abs+1))

            section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
        #     print(section_abst)
        #     print(cur_section+'%i'%(cur_abs+1))
        #     print(i, abs_beg_ind, abs_end_ind)
        except ValueError:
            print('number not found')
            try:
                start_abst=section_abst.index(' — ')#index with in the section where we first see this character - which denotes start of abs

                whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
                title_sect=section_abst[0:start_abst]#the title and author combined
                title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
                title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words
                length_title_lst=' '.join(title_lst)
                auth_sect=section_abst[len(length_title_lst):start_abst]


                sect_abs.append(letter+'%i'%cur_abs)
                abst.append(whole_abs)
                title.append(length_title_lst)
                #auth.append(title_sect) old wrong way of getting author
                auth.append(auth_sect)
            except:
                print("somthing")
            


number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

...

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not found
number not fou

In [ ]:
sect_abs=[]
abst=[]
title=[]
auth=[]
i=1
letter_start_ind = data_list.index('\x0cPoster Session A')
letter_end_ind = data_list.index('\x0cPoster Session C')
a_b_range= range(letter_start_ind,letter_end_ind)
#a_b_range=list(a_b_range)
alpha=['A','B','C','D','E','F']
file = open(data_folder+CNS_files[9], 'r')
data = file.read()
data_list = data.split('\n')
poster_beg_ind = data_list.index('\x0cPoster Session A')

#messing with letter iterations

big_range=letter_end_ind-letter_start_ind


#for cur_section in 

for i in range(big_range):
    try:
        cur_section='A'
        cur_abs=i+1
#         abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
#         abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))
        abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
        abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

        section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
#     print(section_abst)
#     print(cur_section+'%i'%(cur_abs+1))
#     print(i, abs_beg_ind, abs_end_ind)
        start_abst=section_abst.index(' — ')#index with in the section where we first see this character - which denotes start of abs

        whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
        title_sect=section_abst[0:start_abst]#the title and author combined
        title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
        title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words
        length_title_lst=' '.join(title_lst)
        auth_sect=section_abst[len(length_title_lst):start_abst]


        sect_abs.append(cur_section+'%i'%cur_abs)
        abst.append(whole_abs)
        title.append(length_title_lst)
        #auth.append(title_sect) old wrong way of getting author
        auth.append(auth_sect)
    
    except :
        print("next letter")
        except:
            for i in range(big_range):
                cur_section='B'
                cur_abs=i+1
        #         abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
        #         abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))
                abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
                abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

                section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])#entire section including title author and abstract 
        #     print(section_abst)
        #     print(cur_section+'%i'%(cur_abs+1))
        #     print(i, abs_beg_ind, abs_end_ind)
                start_abst=section_abst.index(' — ')#index with in the section where we first see this character - which denotes start of abs

                whole_abs=section_abst[start_abst:len(section_abst)]#the abstract separated from the title and author
                title_sect=section_abst[0:start_abst]#the title and author combined
                title_auth_lst=title_sect.split(' ')#splits them up into list items so that the lamdas will work? have not tried to run without this
                title_lst=[word for word in title_auth_lst if word.isupper()]#takes only all uppercase words
                length_title_lst=' '.join(title_lst)
                auth_sect=section_abst[len(length_title_lst):start_abst]


                sect_abs.append(cur_section+'%i'%cur_abs)
                abst.append(whole_abs)
                title.append(length_title_lst)
                #auth.append(title_sect) old wrong way of getting author
                auth.append(auth_sect)
                except:
                    print('end')

In [10]:
#making a datatable to see how the data fills the table which will help better understand the format i need to put the data into
try_table=pd.DataFrame({'sect_abs':sect_abs,'abst':abst,'title':title,'auth':auth})
try_table



,abst,auth,sect_abs,title
0,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A172,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
1,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A173,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
2,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A174,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
3,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A175,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
4,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A176,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
5,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A177,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
6,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A178,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
7,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A179,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
8,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A180,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...
9,— Fluency with numer- ical symbols is essent...,"Andrew Mattarella-Micke1, Bruce McCandliss1; ...",A181,A171 TASK DEPENDENT NUMERICAL REPRESENTATIONS ...


In [ ]:
alpha=['A','B','C','D','E','F','G','H','I']
number=i+1


In [ ]:
huh=[]
for letter in alpha:
    for i in range(10):
        h=i+1
        you=letter+'%i'%h
        huh.append(you)
   

In [ ]:
# so the below stuff works off of what you can do with the whole_abs cell a couple above. I Think the problem
# with the below cell is that a it might not work on the same priciples that the above one does, like 
# maybe it is working with one simple block
# or B i think that abs_list needs to be initalized to make it so I am only looking at CNS 2016
# once you can make sure you are only working with CNS 2016 then I think that the below should work


In [ ]:
cns_frame=pd.DataFrame({'CNS_file':CNS_file,'start_sect':start_sect,'beg_ind':beg_ind,'end_ind':end_ind})

In [ ]:
so lets get this to work at least for the A's and then we can worry about moving to the next letter.
so im thinking we do a while loop saying while cur_section = A: current abstract = i initialize 
    i to be one outside then have i increment inside

In [ ]:
cns_frame

In [ ]:
cur_section= ['']
alpha= string.ascii_uppercase
alpha_list=[]
for i in alpha:
    alpha_list.append(i)
alpha_list
for j in range(len(alpha_list)):
    while cur_section == alpha_list[j]


In [ ]:
if cur_section letter + curabs number in abs_list.index(cur_section+'%i'%cur_abs):
    do all this appending stuff of the abstract information

In [ ]:
def count(s,value):
    total=0
    for elem in s:
        if elem== value:
            total=total+1
    return total

In [ ]:
import string

In [ ]:
alpha= string.ascii_uppercase
alpha_list=[]
for i in alpha:
    alpha_list.append(i)
alpha_list
    

In [ ]:
#d = {'sect+abs':[cur_section],'abs':[whole_abs],'title':[title_lst],'auth':[title_sect]}
d = {'sect+abs':[],'abs':[],'title':[],'auth':[]}
Df_2016=pd.DataFrame(data=d)
Df_2016

In [ ]:
df_test=pd.DataFrame(columns=['Year','Title','Abstract','Author',])


In [ ]:
#[item for item in title_test_1 if item not in title_lst]

In [ ]:
title_lst=' '.join(title_lst)
title_lst

In [ ]:
whole_abs

In [ ]:
cur_section = 'A'
#cur_section = 'E '
cur_abs = 129
abs_beg_ind = abs_list.index(cur_section+'%i'%cur_abs)
abs_end_ind = abs_list.index(cur_section+'%i'%(cur_abs+1))

#section_abst=' '.join(abs_list[abs_beg_ind:abs_end_ind])
this_sec=abs_list[abs_beg_ind:abs_end_ind]
#title_start=len(abs_list[abs_beg_ind])
this_sec[this_sec.isupper]
#abs_list[title_start:
#whole_title=abs_list[title_start:
#want all the string until I hit a lower case string
#section_abst
# start_abst=section_abst.index(" — ")
# whole_abs=section_abst[start_abst:]
# whole_abs
#abs_list[abs_beg_ind:abs_end_ind]

In [ ]:
#for i in range(len(CNS_files)):
file = open(data_folder+CNS_files[0], 'r')
print(CNS_files[0])
data = file.read()
data_list = data.split('\n')
#abs_start_del= data_list.iloc[" - "]
poster_beg_ind = data_list.index('\x0cPoster Session A')
test_a= data_list[poster_beg_ind:poster_beg_ind+10]
# abs_start_del = data_list.index('\x0c— ')
# if '\x0cPoster Topic Index' in data_list:
#     poster_end_ind = data_list.index('\x0cPoster Topic Index')
# else:
#     poster_end_ind = data_list.index('\x0cAuthor Index')

#     #print(poster_beg_ind, poster_end_ind)
#     print(data_list[poster_beg_ind:poster_beg_ind+7])
#     print('---')
#     print(data_list[poster_beg_ind:abs_start_del])
#test_a

# if testing.find(" — ") == -1:
#     print ("No ' — ' here!")
# else:
#     print ("Found 'is' in the string.")

ahhh= testing.index(" — ")
ahhh


In [ ]:
print(CNS_files[0])
file = open(data_folder+CNS_files[0], 'r')
data = file.read()
data_list = data.split('\n')
print(data_list.index('\x0cPoster Session A'))
data_list[data_list.index('\x0cPoster Session A'):poster_beg_ind+15]
testing=' '.join(data_list[data_list.index('\x0cPoster Session A'):poster_beg_ind+15])
# if testing.find(" — ") == -1: #okay so this is the delimeter for the start of the abstract
#     print ("No ' — ' here!")
# else:
#     print ("Found 'is' in the string.")#this returns bc this is in this chunk of string. 
#so now I want to do something where it groups each abstract section from some point to s

ahhh= testing.index(" — ")
ahhh


In [ ]:
stings='A0 A1 A2 A3 A4 A5 A6'
j=[0,1,2,3,4,5,6]
for i in range(len(strings)):
    if 